# fairness

In [52]:
PATH_PREFIX = '/your/path/to/this/repo/InvDiff/data/'
WANDB=True

## no eiil

In [53]:
template = '''
export MODEL_NAME="CompVis/stable-diffusion-v1-4"
export DATASET_NAME="invariant/datasets/fairness/{dataset_name}"
export PATH_PREFIX={path_prefix}


export LAMBDA=0
export DELTA_TYPE="small"
export SOFT_GROUPER={sg}
export PATH_GROUPER={pg} # "t_g2_w3"
export LEARNING_RATE=1e-04
export BATCH_SIZE=64
export AC_STEPS=1
export LR_SCHEDULER="cosine"
export WARMUP_STEPS=1000
export DELTA_PARAM={dp}
export DELTA_RATIO={dr}
export HARD_GROUPER_NUM=4

accelerate launch --mixed_precision="fp16" src/train_text_to_image.py \\
  --path_prefix=$PATH_PREFIX \\
  --pretrained_model_name_or_path=$MODEL_NAME \\
  --train_data_dir=$DATASET_NAME \\
  --dataset="fairness" \\
  --resolution=512 --center_crop --random_flip \\
  --train_batch_size=${{BATCH_SIZE}} \\
  --gradient_accumulation_steps=${{AC_STEPS}} \\
  --gradient_checkpointing \\
  --max_train_steps=10000 \\
  --learning_rate=${{LEARNING_RATE}} \\
  --max_grad_norm=1 \\
  --lr_scheduler=${{LR_SCHEDULER}} --lr_warmup_steps=${{WARMUP_STEPS}} \\
  --checkpointing_steps=1000 \\
  --eiil=0 \\
  --delta=False \\
  --lambda_value=${{LAMBDA}} \\
  --delta_ratio=${{DELTA_RATIO}} \\
  --delta_type=${{DELTA_TYPE}} \\
  --delta_param=${{DELTA_PARAM}} \\
  --wandb={wandb} \\
  --soft_grouper=${{SOFT_GROUPER}} \\
  --hard_grouper_num=${{HARD_GROUPER_NUM}} \\
  --output_dir="invariant/ckpts/fairness/models/{dataset_name}-noeiil-nodelta-lambda${{LAMBDA}}-delta${{DELTA_RATIO}}-${{DELTA_TYPE}}-deltaparam${{DELTA_PARAM}}-${{SOFT_GROUPER}}-${{HARD_GROUPER_NUM}}-${{PATH_GROUPER}}-bs${{BATCH_SIZE}}-sc${{LEARNING_RATE}}-ac${{AC_STEPS}}-lr${{LR_SCHEDULER}}-wu${{WARMUP_STEPS}}" \\
'''
import os
if not os.path.exists("fairness/diff_dataset_no_eiil"):
    os.makedirs("fairness/diff_dataset_no_eiil")

ratios = [
    [3, 2, 1, 1, 1, 1, 2, 3],
]

target_dataset = [f'split_fairness_{ratio[0]}{ratio[1]}{ratio[2]}{ratio[3]}{ratio[4]}{ratio[5]}{ratio[6]}{ratio[7]}' for ratio in ratios]

sg = "False"
for pg in ["t_g2_w3"]:
    for dp in [0]:
        for dr in [0]:
            for target in target_dataset:
                with open(f"fairness/diff_dataset_no_eiil/run_{dp}_{dr}_sg{sg}_{pg}_tg{target}.sh", "w") as f:
                    f.write(template.format(dp=dp, dr=dr, pg=pg, sg=sg, dataset_name=target, path_prefix=PATH_PREFIX, wandb=WANDB))

## no eiil test

In [54]:
template = '''
export MODEL_NAME="CompVis/stable-diffusion-v1-4"
export DATASET_NAME="invariant/datasets/fairness/split_fairness_11111111"
export PATH_PREFIX={path_prefix}

export LAMBDA=0
export DELTA_TYPE="small"
export SOFT_GROUPER={sg}
export PATH_GROUPER={pg} # "t_g2_w3"
export LEARNING_RATE=1e-04
export BATCH_SIZE=64
export AC_STEPS=1
export LR_SCHEDULER="cosine"
export WARMUP_STEPS=1000
export DELTA_PARAM={dp}
export DELTA_RATIO={dr}
export HARD_GROUPER_NUM=4

text_num=2
sample_num=512

for checkpoint_step in 10000 #5000
do
    python src/test.py \\
        --path_prefix ${{PATH_PREFIX}} \\
        --path_dataset ${{DATASET_NAME}} \\
        --model_name "{dataset_name}-noeiil-nodelta-lambda${{LAMBDA}}-delta${{DELTA_RATIO}}-${{DELTA_TYPE}}-deltaparam${{DELTA_PARAM}}-${{SOFT_GROUPER}}-${{HARD_GROUPER_NUM}}-${{PATH_GROUPER}}-bs${{BATCH_SIZE}}-sc${{LEARNING_RATE}}-ac${{AC_STEPS}}-lr${{LR_SCHEDULER}}-wu${{WARMUP_STEPS}}" \\
        --model_checkpoint_num ${{checkpoint_step}} \\
        --text_num ${{text_num}} \\
        --sample_num ${{sample_num}} \\
        --task_sample True \\
        --save_sample True\\
        --task_lpips False \\
        --task_fid True \\
        --task_recall True \\
        --delta False \\
        --lambda_value ${{LAMBDA}} \\
        --delta_ratio ${{DELTA_RATIO}} \\
        --delta_init0 False \\
        --path_result_root invariant/results/fairness \\
        --path_ckpt invariant/ckpts/fairness/models \\
        --task_clip_score True \\
        --task_bias_score True \\
        --path_classifier invariant/ckpts/fairness/classifier/classifier_race/resnet18-64-0.0001 \\
        --num_class_classifier 4
done
'''
import os

path = "fairness/test/diff_dataset_no_eiil"
if not os.path.exists(path):
    os.makedirs(path)

ratios = [
    [3, 2, 1, 1, 1, 1, 2, 3],
]
target_dataset = [f'split_fairness_{ratio[0]}{ratio[1]}{ratio[2]}{ratio[3]}{ratio[4]}{ratio[5]}{ratio[6]}{ratio[7]}' for ratio in ratios]
sg = "False"
count = 0
for pg in ["t_g2_w3"]:
    for dp in [0]:
        for dr in [0]:
            for target in target_dataset:
                with open(f"{path}/run_small_soft_{dp}_{dr}_sg{sg}_{pg}_tg{target}.sh", "w") as f:
                    f.write(template.format(dp=dp, dr=dr, pg=pg, sg=sg, dataset_name=target, path_prefix=PATH_PREFIX))
                    count += 1
print(f'generate {count} jobs')

    

generate 1 jobs


## eiil-delta hard

In [55]:
template = '''
export MODEL_NAME="CompVis/stable-diffusion-v1-4"
export DATASET_NAME="invariant/datasets/fairness/{dataset_name}"
export PATH_PREFIX={path_prefix}

export LAMBDA={lb}
export DELTA_TYPE="small"
export SOFT_GROUPER={sg}
export PATH_GROUPER={pg} # "t_g2_w3"
export LEARNING_RATE=1e-04
export BATCH_SIZE=64
export AC_STEPS=1
export LR_SCHEDULER="cosine"
export WARMUP_STEPS=1000
export DELTA_PARAM={dp}
export DELTA_RATIO={dr}
export HARD_GROUPER_NUM={hgn}

accelerate launch --mixed_precision="fp16" src/train_text_to_image.py \\
  --path_prefix=$PATH_PREFIX \\
  --pretrained_model_name_or_path=$MODEL_NAME \\
  --train_data_dir=$DATASET_NAME \\
  --dataset="fairness" \\
  --resolution=512 --center_crop --random_flip \\
  --train_batch_size=${{BATCH_SIZE}} \\
  --gradient_accumulation_steps=${{AC_STEPS}} \\
  --gradient_checkpointing \\
  --max_train_steps=10000 \\
  --delta_init0=True \\
  --learning_rate=${{LEARNING_RATE}} \\
  --max_grad_norm=1 \\
  --lr_scheduler=${{LR_SCHEDULER}} --lr_warmup_steps=${{WARMUP_STEPS}} \\
  --checkpointing_steps=1000 \\
  --eiil=1 \\
  --delta=True \\
  --lambda_value=${{LAMBDA}} \\
  --delta_ratio=${{DELTA_RATIO}} \\
  --delta_type=${{DELTA_TYPE}} \\
  --delta_param=${{DELTA_PARAM}} \\
  --wandb={wandb} \\
  --soft_grouper=${{SOFT_GROUPER}} \\
  --hard_grouper_num=${{HARD_GROUPER_NUM}} \\
  --based_unet_path="invariant/ckpts/fairness/models/{dataset_name}-noeiil-nodelta-lambda0-delta0-small-deltaparam0-False-4-t_g2_w3-bs64-sc1e-04-ac1-lrcosine-wu1000/checkpoint-10000" \\
  --output_dir="invariant/ckpts/fairness/models/{dataset_name}-eiil-delta-lambda${{LAMBDA}}-delta${{DELTA_RATIO}}-${{DELTA_TYPE}}-deltaparam${{DELTA_PARAM}}-${{SOFT_GROUPER}}-${{HARD_GROUPER_NUM}}-${{PATH_GROUPER}}-bs${{BATCH_SIZE}}-sc${{LEARNING_RATE}}-ac${{AC_STEPS}}-lr${{LR_SCHEDULER}}-wu${{WARMUP_STEPS}}" \\
'''
import os
if not os.path.exists("fairness/diff_dataset_hard"):
    os.makedirs("fairness/diff_dataset_hard")

ratios = [
    [3, 2, 1, 1, 1, 1, 2, 3],
]

target_dataset = [f'split_fairness_{ratio[0]}{ratio[1]}{ratio[2]}{ratio[3]}{ratio[4]}{ratio[5]}{ratio[6]}{ratio[7]}' for ratio in ratios]
sg = "False"

candidate_lb = [1, 10, 50, 100]
candidate_lb = [20, 30, 40]
candidate_lb = [0.1, 0.5]
candidate_lb = [1]

hard_group_num = [8]


# pretrain
for hgn in hard_group_num:
    for lb in candidate_lb:
        for pg in ["t_g2_w3"]:
            for dp in [0]:
                for dr in [0.4, 0.6, 0.8, 0.9]:
                    for target in target_dataset:
                        with open(f"fairness/diff_dataset_hard/run_{dp}_{dr}_sg{sg}_{pg}_tg{target}_lb{lb}_hgn{hgn}.sh", "w") as f:
                            f.write(template.format(dp=dp, dr=dr, pg=pg, sg=sg, dataset_name=target, lb=lb, hgn=hgn, path_prefix=PATH_PREFIX, wandb=WANDB))


## eiil-delta soft

In [56]:
template = '''
export MODEL_NAME="CompVis/stable-diffusion-v1-4"
export DATASET_NAME="invariant/datasets/fairness/{dataset_name}"
export PATH_PREFIX={path_prefix}

export LAMBDA={lb}
# export DELTA_TYPE="small"
export DELTA_TYPE="pretrain"
export SOFT_GROUPER={sg}
export PATH_GROUPER={pg} # "t_g2_w3"
export LEARNING_RATE=1e-04
export BATCH_SIZE=64
export AC_STEPS=1
export LR_SCHEDULER="cosine"
export WARMUP_STEPS=1000
export DELTA_PARAM={dp}
export DELTA_RATIO={dr}
export HARD_GROUPER_NUM={hgn}

accelerate launch --mixed_precision="fp16" src/train_text_to_image.py \\
    --path_prefix=$PATH_PREFIX \\
  --pretrained_model_name_or_path=$MODEL_NAME \\
  --train_data_dir=$DATASET_NAME \\
  --dataset="fairness" \\
  --resolution=512 --center_crop --random_flip \\
  --train_batch_size=${{BATCH_SIZE}} \\
  --gradient_accumulation_steps=${{AC_STEPS}} \\
  --gradient_checkpointing \\
  --max_train_steps=10000 \\
  --learning_rate=${{LEARNING_RATE}} \\
  --max_grad_norm=1 \\
  --lr_scheduler=${{LR_SCHEDULER}} --lr_warmup_steps=${{WARMUP_STEPS}} \\
  --checkpointing_steps=1000 \\
  --eiil=1 \\
  --delta=True \\
  --lambda_value=${{LAMBDA}} \\
  --delta_ratio=${{DELTA_RATIO}} \\
  --delta_type=${{DELTA_TYPE}} \\
  --delta_param=${{DELTA_PARAM}} \\
  --wandb={wandb} \\
  --soft_grouper=${{SOFT_GROUPER}} \\
  --hard_grouper_num=${{HARD_GROUPER_NUM}} \\
  --based_unet_path="invariant/ckpts/fairness/models/{dataset_name}-noeiil-nodelta-lambda0-delta0-small-deltaparam0-False-4-t_g2_w3-bs64-sc1e-04-ac1-lrcosine-wu1000/checkpoint-10000" \\
  --path_grouper="invariant/ckpts/fairness/groupers/groupers/{dataset_name}/weights_${{PATH_GROUPER}}.pkl" \\
  --output_dir="invariant/ckpts/fairness/{dataset_name}-eiil-delta-lambda${{LAMBDA}}-delta${{DELTA_RATIO}}-${{DELTA_TYPE}}-deltaparam${{DELTA_PARAM}}-${{SOFT_GROUPER}}-${{HARD_GROUPER_NUM}}-${{PATH_GROUPER}}-bs${{BATCH_SIZE}}-sc${{LEARNING_RATE}}-ac${{AC_STEPS}}-lr${{LR_SCHEDULER}}-wu${{WARMUP_STEPS}}" \\
  
'''
import os
if not os.path.exists("fairness/diff_dataset_soft"):
    os.makedirs("fairness/diff_dataset_soft")


ratios = [
    [3, 2, 1, 1, 1, 1, 2, 3],
]
target_dataset = [f'split_fairness_{ratio[0]}{ratio[1]}{ratio[2]}{ratio[3]}{ratio[4]}{ratio[5]}{ratio[6]}{ratio[7]}' for ratio in ratios]

sg = "True"

candidate_lb = [1, 10, 50, 100]
candidate_lb = [20, 30, 40]
candidate_lb = [1]


hard_group_num = [2, 4]
hard_group_num = [8]
for hgn in hard_group_num:
    for lb in candidate_lb:
        for pg in ["t_g8_w3", "t_g8_w5", "t_g8_w0", "t_g16_w0", "t_g16_w3", "t_g16_w5"]:
            for dp in [0]:
                for dr in [0.4, 0.6, 0.8, 0.9]:
                    for target in target_dataset:
                        with open(f"fairness/diff_dataset_soft/run_{dp}_{dr}_sg{sg}_{pg}_tg{target}_lb{lb}_hgn{hgn}.sh", "w") as f:
                            f.write(template.format(dp=dp, dr=dr, pg=pg, sg=sg, dataset_name=target, lb=lb, hgn=hgn, path_prefix=PATH_PREFIX, wandb=WANDB))

## eiil-delta test

In [59]:
template = '''
export MODEL_NAME="CompVis/stable-diffusion-v1-4"
export DATASET_NAME="invariant/datasets/fairness/split_fairness_11111111"
export PATH_PREFIX={path_prefix}

export LAMBDA={lb}
export DELTA_TYPE="small"
export SOFT_GROUPER={sg}
export PATH_GROUPER={pg} # "t_g2_w3"
export LEARNING_RATE=1e-04
export BATCH_SIZE=64
export AC_STEPS=1
export LR_SCHEDULER="cosine"
export WARMUP_STEPS=1000
export DELTA_PARAM={dp}
export DELTA_RATIO={dr}
export HARD_GROUPER_NUM={hgn}

text_num=2
sample_num=512

for checkpoint_step in 10000 #5000 3000
do
    python src/test.py \\
        --path_prefix ${{PATH_PREFIX}} \\
        --path_dataset ${{DATASET_NAME}} \\
        --model_name "{dataset_name}-eiil-delta-lambda${{LAMBDA}}-delta${{DELTA_RATIO}}-${{DELTA_TYPE}}-deltaparam${{DELTA_PARAM}}-${{SOFT_GROUPER}}-${{HARD_GROUPER_NUM}}-${{PATH_GROUPER}}-bs${{BATCH_SIZE}}-sc${{LEARNING_RATE}}-ac${{AC_STEPS}}-lr${{LR_SCHEDULER}}-wu${{WARMUP_STEPS}}" \\
        --model_checkpoint_num ${{checkpoint_step}} \\
        --text_num ${{text_num}} \\
        --sample_num ${{sample_num}} \\
        --task_sample True \\
        --save_sample True\\
        --task_lpips False \\
        --task_fid True \\
        --task_recall True \\
        --delta True \\
        --lambda_value ${{LAMBDA}} \\
        --delta_ratio ${{DELTA_RATIO}} \\
        --delta_init0 False \\
        --path_result_root invariant/results/fairness \\
        --path_ckpt invariant/ckpts/fairness/models \\
        --task_clip_score True \\
        --task_bias_score True \\
        --path_classifier invariant/ckpts/fairness/classifier/classifier_race/resnet18-64-0.0001 \\
        --num_class_classifier 4
done
'''
import os

path = "fairness/test/diff_dataset"
if not os.path.exists(path):
    os.makedirs(path)


ratios = [
    [3, 2, 1, 1, 1, 1, 2, 3],
    ]
candidate_lb = [1]
hard_group_num = [8]
target_dataset = [f'split_fairness_{ratio[0]}{ratio[1]}{ratio[2]}{ratio[3]}{ratio[4]}{ratio[5]}{ratio[6]}{ratio[7]}' for ratio in ratios]
sg = "False"
for hgn in hard_group_num:
    for lb in candidate_lb:
        for pg in ["t_g2_w3"]:
            for dp in [0]:
                for dr in [0.4, 0.6, 0.8, 0.9]:
                    for target in target_dataset:
                        with open(f"{path}/run_{dp}_{dr}_sg{sg}_{pg}_tg{target}_lb{lb}_hgn{hgn}.sh", "w") as f:
                            f.write(template.format(dp=dp, dr=dr, pg=pg, sg=sg, dataset_name=target, lb=lb, hgn=hgn, path_prefix=PATH_PREFIX))

sg = "True"
for hgn in hard_group_num:
    for lb in candidate_lb:
        for pg in ["t_g8_w0", "t_g8_w5"]:
            for dp in [0]:
                for dr in [0.4, 0.6, 0.8, 0.9]:
                    for target in target_dataset:
                        with open(f"{path}/run_{dp}_{dr}_sg{sg}_{pg}_tg{target}_lb{lb}_hgn{hgn}.sh", "w") as f:
                            f.write(template.format(dp=dp, dr=dr, pg=pg, sg=sg, dataset_name=target, lb=lb, hgn=hgn, path_prefix=PATH_PREFIX))
    

## grouper

In [58]:
template = '''
export MODEL_NAME="CompVis/stable-diffusion-v1-4"
export DATASET_NAME="invariant/datasets/fairness/{dataset_name}"

echo "start {dataset_name} samples"
accelerate launch --mixed_precision="fp16"  src/group/generate_train_text_to_image.py \\
  --path_prefix={path_prefix} \\
  --pretrained_model_name_or_path=$MODEL_NAME \\
  --train_data_dir=$DATASET_NAME \\
  --use_ema \\
  --resolution=512 --center_crop --random_flip \\
  --train_batch_size=16 \\
  --gradient_accumulation_steps=1 \\
  --gradient_checkpointing \\
  --max_train_steps=15000 \\
  --learning_rate=1e-05 \\
  --max_grad_norm=1 \\
  --lr_scheduler="constant" --lr_warmup_steps=0 \\
  --output_dir="invariant/ckpts/fairness/groupers/samples/{dataset_name}/" \\
  --based_unet_path="invariant/ckpts/fairness/models/{dataset_name}-noeiil-nodelta-lambda0-delta0-small-deltaparam0-False-4-t_g2_w3-bs64-sc1e-04-ac1-lrcosine-wu1000/checkpoint-10000" 

echo "finish {dataset_name} samples"


echo "start {dataset_name} groupers group_num={group_num} loss_not_zero_weight={loss_not_zero_weight}"
python src/group/train_group.py \
    --path_embedding_file="invariant/ckpts/fairness/groupers/samples/{dataset_name}/samples_t.pkl" \
    --path_prefix="{path_prefix}" \
    --path_groupers="invariant/ckpts/fairness/groupers/groupers/{dataset_name}" \
    --group_num={group_num} \
    --loss_not_zero_weight={loss_not_zero_weight} \

echo "finish {dataset_name} groupers"
'''

template2 = '''
export MODEL_NAME="CompVis/stable-diffusion-v1-4"
export DATASET_NAME="invariant/datasets/fairness/{dataset_name}"
echo "start {dataset_name} groupers group_num={group_num} loss_not_zero_weight={loss_not_zero_weight}"
python src/group/train_group.py \
    --path_embedding_file="invariant/ckpts/fairness/groupers/samples/{dataset_name}/samples_t.pkl" \
    --path_prefix="{path_prefix}" \
    --path_groupers="invariant/ckpts/fairness/groupers/groupers/{dataset_name}" \
    --group_num={group_num} \
    --loss_not_zero_weight={loss_not_zero_weight} \

echo "finish {dataset_name} groupers"

'''

script_path = "fairness/grouper/samples"
import os
if not os.path.exists(script_path):
    os.makedirs(script_path)

log_path = "fairness/grouper/log"
if not os.path.exists(log_path):
    os.makedirs(log_path)

ratios = [
    [3, 2, 1, 1, 1, 1, 2, 3],
]

target_dataset = [f'split_fairness_{ratio[0]}{ratio[1]}{ratio[2]}{ratio[3]}{ratio[4]}{ratio[5]}{ratio[6]}{ratio[7]}' for ratio in ratios]
with open(f"{script_path}/run_all.sh", "w") as f:
    pass
count = 0
for group_num in [8,16]:
    for loss_not_zero_weight in [0,3,5]:
        for target in target_dataset:
            with open(f"{script_path}/run_{target}_g{group_num}_w{loss_not_zero_weight}.sh", "w") as f:
                if loss_not_zero_weight == 0 and group_num == 8:
                    f.write(template.format(dataset_name=target, group_num=group_num, loss_not_zero_weight=loss_not_zero_weight, path_prefix=PATH_PREFIX))
                else:
                    f.write(template2.format(dataset_name=target, group_num=group_num, loss_not_zero_weight=loss_not_zero_weight, path_prefix=PATH_PREFIX))
            
            with open(f"{script_path}/run_all.sh", "a") as f:
                f.write(f'echo "running {target} g{group_num} w{loss_not_zero_weight}"\n')
                f.write(f'bash scripts/{script_path}/run_{target}_g{group_num}_w{loss_not_zero_weight}.sh >> scripts/{log_path}/run_{target}_g{group_num}_w{loss_not_zero_weight}.log 2>&1 \n')
            count += 1

print(f'generate {count} jobs')

generate 6 jobs
